In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv', encoding='gb18030')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
def make_label(df):
    df["sentiment"] = df["star"].apply(lambda x: 1 if x>3 else 0)

In [ ]:
make_label(df)

In [ ]:
df.head()

In [ ]:
X = df[['comment']]
y = df.sentiment

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X.head()

In [ ]:
import jieba

In [ ]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [ ]:
X['cutted_comment'] = X.comment.apply(chinese_word_cut)

In [ ]:
X.cutted_comment[:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [ ]:
stop_words_file = "stopwordsHIT.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [ ]:
stopwords[-10:]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect = CountVectorizer()

In [ ]:
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())

In [ ]:
term_matrix.head()

In [ ]:
term_matrix.shape

In [ ]:
vect = CountVectorizer(stop_words=frozenset(stopwords))

In [ ]:
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())

In [ ]:
term_matrix.head()

In [ ]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。

In [ ]:
vect = CountVectorizer(max_df = max_df, 
                       min_df = min_df, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))

In [ ]:
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())

In [ ]:
term_matrix.head()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(vect, nb)

In [ ]:
pipe.steps

In [ ]:
from sklearn.cross_validation import cross_val_score
cross_val_score(pipe, X_train.cutted_comment, y_train, cv=5, scoring='accuracy').mean()

In [ ]:
pipe.fit(X_train.cutted_comment, y_train)

In [ ]:
pipe.predict(X_test.cutted_comment)

In [ ]:
y_pred = pipe.predict(X_test.cutted_comment)

In [ ]:
from sklearn import metrics

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
from snownlp import SnowNLP
def get_sentiment(text):
    return SnowNLP(text).sentiments

In [ ]:
y_pred_snownlp = X_test.comment.apply(get_sentiment)

In [ ]:
y_pred_snownlp_normalized = y_pred_snownlp.apply(lambda x: 1 if x>0.5 else 0)

In [ ]:
y_pred_snownlp_normalized[:5]

In [ ]:
metrics.accuracy_score(y_test, y_pred_snownlp_normalized)

In [ ]:
metrics.confusion_matrix(y_test, y_pred_snownlp_normalized)